In [1]:
import os
import gc
import time
import math
import warnings

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from joblib import delayed, Parallel

warnings.filterwarnings('ignore')

# 计算特征的函数

In [2]:
def cal_bollingerBands(data):
    """
    计算每个tick的布林带，布林带上下轨与移动平均线的标准差倍数默认为2，窗口[5, 10, 20, 40]
    Input:
        data: 一只股票一天的snapshot数据
    Output:
        ret: 单只股票每个tick的布林带上轨、中轨、下轨的numpy数组
                        (4个窗口分别对应上轨、中轨、下轨，总共12列)
        time: 时间戳数组
    """
    time = pd.to_datetime(data['time'])
    tick_prices = data['n_close']
    
    windows=[5, 10, 20, 40]
    ret = np.zeros((tick_prices.shape[0], 3*len(windows)))
    ret[:] = np.nan
    
    num_std=2

    for idx, window in enumerate(windows):
        rolling_mean = tick_prices.rolling(window=window, min_periods=window).mean()
        rolling_std = tick_prices.rolling(window=window, min_periods=window).std()

        # 计算布林带上下轨
        upper_band = rolling_mean + num_std * rolling_std
        lower_band = rolling_mean - num_std * rolling_std

        # 将布林带上轨、中轨、下轨加入结果数组
        ret[:, idx*3] = upper_band.values
        ret[:, idx*3 + 1] = rolling_mean.values
        ret[:, idx*3 + 2] = lower_band.values

    return ret, time

In [3]:
# data = pd.read_csv('./data/train/snapshot_sym0_date0_am.csv', index_col=0)
# ret, time1 = cal_bollingerBands(data)

In [4]:
def cal_cmo(data):
    """
    计算Chande Momentum Oscillator (CMO)因子
    输入：
        data: 一只股票一天的snapshot数据
    输出：
        ret：包含5、10、20、40窗口大小的CMO因子值的numpy数组
    """
    close_prices = data['n_close'].values
    windows = [5, 10, 20, 40]
    ret = np.zeros((len(close_prices), len(windows)))
    ret[:] = np.nan
    
    for idx, window in enumerate(windows):
        up_changes = np.zeros(len(close_prices))
        down_changes = np.zeros(len(close_prices))

        for i in range(window, len(close_prices)):
            up_sum = np.sum(close_prices[i-window+1:i+1][close_prices[i-window+1:i+1] > close_prices[i-window+1]])
            down_sum = np.sum(close_prices[i-window+1:i+1][close_prices[i-window+1:i+1] < close_prices[i-window+1]])
            up_changes[i-1] = up_sum
            down_changes[i-1] = down_sum
            
        ret[:window-1, idx] = np.nan
        ret[window-1:, idx] = ((up_changes[window-1:]-down_changes[window-1:]) / (up_changes[window-1:]+down_changes[window-1:]))

    return ret

In [5]:
def cal_mom(data):
    """
    计算窗口内的Momentum值，窗口选1、5、10、20、40
    Input:
        data: 一只股票一天的snapshot数据
    Output:
        ret: 单只股票每个tick的Momentum因子，包含1、5、10、20、40窗口
    """
    close_prices = data['n_close'].values

    windows = [1, 5, 10, 20, 40]
    ret = np.zeros((close_prices.shape[0], len(windows)))
    ret[:] = np.nan

    for i, window in enumerate(windows):
        ret[window:, i] = close_prices[window:] - close_prices[:-window]

    return ret

In [6]:
def cal_roc(data):
    """
    计算ROC (Rate of Change)因子,收盘价相对前window期价格的相对变化率，窗口选1, 5, 10, 20, 40
    Input:
        data: 一只股票一天的snapshot数据
    Output:
        ret: 收盘价的ROC因子，包含1、5、10、20、40窗口的结果
    """
    close_prices = data['n_close'].values
    windows = [1, 5, 10, 20, 40]
    ret = np.zeros((close_prices.shape[0], len(windows)))
    ret[:] = np.nan
    
    for i, window in enumerate(windows):
        roc_values = close_prices[window:] / close_prices[:-window] - 1
        ret[window:, i] = roc_values

    return ret

In [7]:
def cal_rsi(data):
    """
    计算相对强弱指数（RSI）
    Input:
        data: 一只股票一天的snapshot数据
    Output:
        ret : RSI指标数组，包含RSI值
    """
    windows = [5, 10, 20, 40]
    close_prices = data['n_close']
    ret = np.zeros((close_prices.shape[0], len(windows)))
    ret[:] = np.nan
    
    price_change = close_prices.diff()

    positive_change = price_change.where(price_change > 0, 0)
    negative_change = -price_change.where(price_change < 0, 0)

    for i, window in enumerate(windows):
        avg_gain = positive_change.rolling(window=window, min_periods=window).mean()
        avg_loss = negative_change.rolling(window=window, min_periods=window).mean()
        relative_strength = avg_gain / avg_loss
        rsi = 100 - (100 / (1 + relative_strength))
        ret[:, i] = rsi
        
    return ret

In [8]:
def cal_mktIntensity(data):
    """
    计算市场紧密度因子，2*5
    报价差因子：qs = (n_ask{i} - n_bid{i}) / (0.5 * (n_ask{i} + n_bid{i}))
    有效价差因子：es = 2 * (n_close - 0.5 * (n_ask{i} + n_bid{i})) / (0.5 * (n_ask{i} + n_bid{i}))
    Input:
        data: 一只股票一天的snapshot数据
    Output:
        ret: 单只股票一天的因子值 (qs和es)
    """
    ret = np.zeros((data.shape[0], 10))
    time = data['time'].values
    
    n_ask = data[['n_ask1', 'n_ask2', 'n_ask3', 'n_ask4', 'n_ask5']].values
    n_bid = data[['n_bid1', 'n_bid2', 'n_bid3', 'n_bid4', 'n_bid5']].values
    n_close = data['n_close'].values

    qs = (n_ask - n_bid) / (0.5 * (n_ask + n_bid))
    es = 2 * (n_close[:, np.newaxis] - 0.5 * (n_ask + n_bid)) / (0.5 * (n_ask + n_bid))

    ret[:, :5] = qs
    ret[:, 5:] = es

    return ret

In [9]:
def cal_elatricityTrading(data):
    """
    计算elatricityTrading交易弹性因子，elatricityTrading = 成交量变化率 / 收盘价变化率
    Input:
        data: 一只股票一天的snapshot数据
    Output:
        ret: 单只股票一天的因子值 (elatricityTrading)
    """
    ret = np.zeros((data.shape[0], 1))
    time = data['time'].values
    
    amount = data['amount_delta'].values
    n_close = data['n_close'].values

    amount_delta = amount[1:] / amount[:-1] - 1  # 成交量变化率
    close_delta = n_close[1:] / n_close[:-1] - 1  # 收盘价变化率

    elatricityTrading = amount_delta / close_delta

    ret[1:, 0] = elatricityTrading
    ret[0, 0] = np.nan

    return ret

In [10]:
def cal_volatility(data):
    """
    计算窗口内的波动率，窗口选5, 10, 20, 40
    Input:
        data: 一只股票一天的snapshot数据
    Output:
        ret: 单只股票每个tick的波动率，包含5、10、20、40窗口的结果
    """
    time = data['time'].values
    tick_prices = data['n_close']

    # 计算收益率
    returns = (tick_prices / tick_prices.shift(1)) - 1

    windows = [5, 10, 20, 40]
    ret = np.zeros((tick_prices.shape[0], len(windows)))

    for idx, window in enumerate(windows):
        # 使用rolling计算波动率，std()默认自动考虑NaN值
        volatility = returns.rolling(window).std() * np.sqrt(252)
        ret[:, idx] = volatility.values
        
    return ret

In [11]:
def cal_mpc(data):
    """
    计算MPC（Midpoint Price Change）因子，mid-price在一个窗口内的变化率，窗口选5, 10, 20, 40
    Input:
        data: 一只股票一天的snapshot数据
    Output:
        ret: 单只股票一天的因子值
    """
    time = data['time'].values
    
    n_midprice = data['n_midprice'].values

    windows = [5, 10, 20, 40]
    ret = np.zeros((data.shape[0], len(windows)))
    
    for idx, window in enumerate(windows):
        mpc_values = (n_midprice[window:] - n_midprice[:-window]) / n_midprice[:-window]
        # 对于窗口大小之前的数据，填NaN
        mpc_values = np.concatenate((np.full(window, np.nan), mpc_values))
        
        ret[:, idx] = mpc_values
    
    return ret

In [12]:
def cal_timeEncoding(data):
    """
    构造时、分、秒以及特定时间段的encoding特征
    Input:
        data: 一只股票一天的snapshot数据
    Output:
        ret: 包含时、分、秒和特定时间段的encoding特征的numpy数组
    """
    time = pd.to_datetime(data['time'])
    
    # 提取时、分、秒
    hour = time.dt.hour
    minute = time.dt.minute
    second = time.dt.second
    
    # 创建特定时间段的编码特征
    trading_time_intervals = [
        (pd.to_datetime('09:30:00').time(), pd.to_datetime('10:00:00').time()),
        (pd.to_datetime('10:00:00').time(), pd.to_datetime('10:30:00').time()),
        (pd.to_datetime('10:30:00').time(), pd.to_datetime('11:00:00').time()),
        (pd.to_datetime('11:00:00').time(), pd.to_datetime('11:30:00').time()),
        (pd.to_datetime('13:00:00').time(), pd.to_datetime('13:30:00').time()),
        (pd.to_datetime('13:30:00').time(), pd.to_datetime('14:00:00').time()),
        (pd.to_datetime('14:00:00').time(), pd.to_datetime('14:30:00').time()),
        (pd.to_datetime('14:30:00').time(), pd.to_datetime('15:00:00').time())
    ]
    
    trading_intervals_encoding = np.zeros((len(data),), dtype=int)
    for idx, (start_time, end_time) in enumerate(trading_time_intervals):
        mask = (time.dt.time >= start_time) & (time.dt.time < end_time)
        trading_intervals_encoding[mask] = idx
    
    # 将编码后的时、分、秒和特定时间段的特征组合成一个numpy数组
    ret = trading_intervals_encoding.reshape(-1,1)
    hour_array = hour.values.reshape(-1, 1)
    minute_array = minute.values.reshape(-1, 1)
    second_array = second.values.reshape(-1, 1)
    ret = np.concatenate((ret, hour_array, minute_array, second_array), axis=1)
    
    return ret

# 扩充特征

In [13]:
def cal_extra_fea(df_original_fea, window, fac_to_remain=None, stadardize=False):
    '''
    df_original_fea: pd.Series, index为date_id和security_id, name为因子名称
    window: list
    fac_to_remain: list, 指保留的因子名称
     '''
    f = df_original_fea.name
    fea_wide = df_original_fea.unstack()
    feature = pd.DataFrame(index=df_original_fea.index)
    # 是否进行量纲处理
    if stadardize:
        df_stadardize = df_original_fea
    else:
        df_stadardize = 1
    for w in window:            
        # mom
        fname = f'{f}_mom{w}'
        if fac_to_remain is None or fname in fac_to_remain:
            shifted_fea_wide = fea_wide.shift(w)
            divisor = np.where(shifted_fea_wide != 0, shifted_fea_wide, np.nan)
            fea_tmp = (fea_wide - shifted_fea_wide) / divisor
            feature[fname] = fea_tmp.stack(dropna=False)

        # mean
        fname = f'{f}_mean{w}'
        if fac_to_remain is None or fname in fac_to_remain:
            fea_tmp = fea_wide.rolling(w, min_periods=w).mean()
            feature[fname] = fea_tmp.stack(dropna=False)
        # std
        fname = f'{f}_std{w}'
        if fac_to_remain is None or fname in fac_to_remain:
            fea_tmp = fea_wide.rolling(w, min_periods=w).std()
            feature[fname] = fea_tmp.stack(dropna=False)
        # max
        fname = f'{f}_max{w}'
        if fac_to_remain is None or fname in fac_to_remain:
            fea_tmp = fea_wide.rolling(w, min_periods=w).max()
            feature[fname] = fea_tmp.stack(dropna=False) / df_stadardize
        # min
        fname = f'{f}_min{w}'
        if fac_to_remain is None or fname in fac_to_remain:
            fea_tmp = fea_wide.rolling(w, min_periods=w).min()
            feature[fname] = fea_tmp.stack(dropna=False) / df_stadardize
        # skew
        fname = f'{f}_skew{w}'
        if fac_to_remain is None or fname in fac_to_remain:
            fea_tmp = fea_wide.rolling(w, min_periods=w).skew()
            feature[fname] = fea_tmp.stack(dropna=False) / df_stadardize
        # kurt
        fname = f'{f}_kurt{w}'
        if fac_to_remain is None or fname in fac_to_remain:
            fea_tmp = fea_wide.rolling(w, min_periods=w).kurt()
            feature[fname] = fea_tmp.stack(dropna=False) / df_stadardize
        # qtlu
        fname = f'{f}_qtlu{w}'
        if fac_to_remain is None or fname in fac_to_remain:
            fea_tmp = fea_wide.rolling(w, min_periods=w).quantile(0.8)
            feature[fname] = fea_tmp.stack(dropna=False) / df_stadardize
        # qtld
        fname = f'{f}_qtld{w}'
        if fac_to_remain is None or fname in fac_to_remain:
            fea_tmp = fea_wide.rolling(w, min_periods=w).quantile(0.2)
            feature[fname] = fea_tmp.stack(dropna=False) / df_stadardize
        # rank
        fname = f'{f}_rank{w}'
        if fac_to_remain is None or fname in fac_to_remain:
            fea_tmp = fea_wide.rolling(w, min_periods=w).rank(pct=True)
            feature[fname] = fea_tmp.stack(dropna=False)
        # IMAX
        fname = f'{f}_imax{w}'
        findMaxIdx  = lambda series: series.shape[0] - series.reset_index(drop=True).idxmax()
        if fac_to_remain is None or fname in fac_to_remain:
            fea_tmp = fea_wide.rolling(w, min_periods=w).apply(findMaxIdx) / w
            feature[fname] = fea_tmp.stack(dropna=False)
        # IMIN
        fname = f'{f}_imin{w}'
        findMinIdx  = lambda series: series.shape[0] - series.reset_index(drop=True).idxmin()
        if fac_to_remain is None or fname in fac_to_remain:
            fea_tmp = fea_wide.rolling(w, min_periods=w).apply(findMinIdx) / w
            feature[fname] = fea_tmp.stack(dropna=False)
        
    return feature

# 生成train的特征

In [14]:
col_org = pd.read_csv(f'./data/train/snapshot_sym0_date0_am.csv', index_col=0).columns.to_list()
len(col_org[3:-5])

23

In [15]:
29 + 23*12*4 + 53 + 49*12*4 + 2
# timeEncoding_fea 没扩充特征
# +2是把index reset了

3540

In [14]:
def gen_factor_train(date, code):
    # 原始特征
    col_org = pd.read_csv(f'./data/train/snapshot_sym0_date0_am.csv', index_col=0).columns.to_list()
    # 生成的特征
    bollinger_fea = [f'{label}Bollinger{i}' for i in [5, 10, 20, 40] for label in ['upper', 'middle', 'lower']]
    cmo_fea = [f'com{i}' for i in [5, 10, 20, 40]]
    mom_fea = [f'mom{i}' for i in [1, 5, 10, 20, 40]]
    roc_fea = [f'roc{i}' for i in [1, 5, 10, 20, 40]]
    rsi_fea = [f'rsi{i}' for i in [5, 10, 20, 40]]
    mktIntensity_fea = [f'qs{i}' for i in range(1,6)] + [f'es{i}' for i in range(1,6)]
    elatricityTrading_fea = ['elatricityTrading']
    volatility_fea = [f'volatility{i}' for i in [5, 10, 20, 40]]
    mpc_fea = [f'mpc{i}' for i in [5, 10, 20, 40]]
    timeEncoding_fea = ['timeEncoding','hour','minute','second']
    # 需要被扩充的 生成的特征
    col_gen = bollinger_fea + cmo_fea + mom_fea + roc_fea + rsi_fea + mktIntensity_fea + elatricityTrading_fea + volatility_fea + mpc_fea
    # 算子
    operators = ['mom', 'mean', 'std', 'max', 'min', 'skew', 'kurt', 'qtlu', 'qtld', 'rank', 'imax', 'imin']
    # 扩充特征——原始的特征（不含sym,date,time,label）
    fea_org_expanded = [f'{f}_{op}{w}' for f in col_org[3:-5] for op in operators for w in [5, 10, 20, 40]]
    # 扩充特征——生成的特征    
    fea_gen_expanded = [f'{f}_{op}{w}' for f in col_gen for op in operators for w in [5, 10, 20, 40]]
    # 所有被扩充的特征
    fac_to_remain = fea_org_expanded + fea_gen_expanded
    
    window = [5,10,20,40]
    
    # 处理csv不存在的情况
    df_am_all = pd.DataFrame()
    df_pm_all = pd.DataFrame()
    
    am_file = f'./data/train/snapshot_sym{code}_date{date}_am.csv'
    isExists = os.path.exists(am_file)
    if isExists:
        data_am = pd.read_csv(am_file, index_col = 0)
        factor_am0, time_am = cal_bollingerBands(data_am)
        factor_am1 = cal_cmo(data_am)
        factor_am2 = cal_mom(data_am)
        factor_am3 = cal_roc(data_am)
        factor_am4 = cal_rsi(data_am)
        factor_am5 = cal_mktIntensity(data_am)
        factor_am6 = cal_elatricityTrading(data_am)
        factor_am7  = cal_volatility(data_am)
        factor_am8  = cal_mpc(data_am)
        factor_am9 = cal_timeEncoding(data_am)
        
        factor_am = np.concatenate([factor_am0, factor_am1, factor_am2, factor_am3, factor_am4, factor_am5, factor_am6, factor_am7, factor_am8, factor_am9], axis = 1)
        factor_am = np.concatenate([data_am.values, factor_am],axis=1)
        
        df_am = pd.DataFrame(factor_am, columns=col_org+col_gen+timeEncoding_fea)
        df_am = df_am.set_index(['time','sym'])
        
        df_am_expanded = pd.DataFrame(index=df_am.index)
        for fea in df_am.columns:
            tmp = cal_extra_fea(df_am[fea], window=window, fac_to_remain=fac_to_remain)
            df_am_expanded = pd.concat([df_am_expanded,tmp], axis=1)
        df_am_all = pd.concat([df_am, df_am_expanded], axis=1)
        
        # 释放内存
        del data_am, factor_am, factor_am0, factor_am1, factor_am2, factor_am3, factor_am4, factor_am5, factor_am6, factor_am7, factor_am8, factor_am9
        gc.collect()
        
    pm_file = f'./data/train/snapshot_sym{code}_date{date}_pm.csv'
    isExists = os.path.exists(pm_file)
    if isExists:
        data_pm = pd.read_csv(pm_file, index_col=0)
        factor_pm0, time_pm = cal_bollingerBands(data_pm)
        factor_pm1 = cal_cmo(data_pm)
        factor_pm2 = cal_mom(data_pm)
        factor_pm3 = cal_roc(data_pm)
        factor_pm4 = cal_rsi(data_pm)
        factor_pm5 = cal_mktIntensity(data_pm)
        factor_pm6 = cal_elatricityTrading(data_pm)
        factor_pm7  = cal_volatility(data_pm)
        factor_pm8  = cal_mpc(data_pm)
        factor_pm9 = cal_timeEncoding(data_pm)
        
        factor_pm = np.concatenate([factor_pm0, factor_pm1, factor_pm2, factor_pm3, factor_pm4, factor_pm5, factor_pm6, factor_pm7, factor_pm8, factor_pm9], axis = 1)
        factor_pm = np.concatenate([data_pm.values, factor_pm],axis=1)
        
        df_pm = pd.DataFrame(factor_pm, columns=col_org+col_gen+timeEncoding_fea)
        df_pm = df_pm.set_index(['time','sym'])
        
        df_pm_expanded = pd.DataFrame(index=df_pm.index)
        for fea in df_pm.columns:
            tmp = cal_extra_fea(df_pm[fea], window=window, fac_to_remain=fac_to_remain)
            df_pm_expanded = pd.concat([df_pm_expanded,tmp], axis=1)
        df_pm_all = pd.concat([df_pm, df_pm_expanded], axis=1)
        
        # 释放内存
        del data_pm, factor_pm, factor_pm0, factor_pm1, factor_pm2, factor_pm3, factor_pm4, factor_pm5, factor_pm6, factor_pm7, factor_pm8, factor_pm9
        gc.collect()

    factor = pd.concat([df_am_all, df_pm_all])

    return factor.reset_index()

In [17]:
num = 64
results_train_1 = Parallel(n_jobs=-1)(delayed(gen_factor_train)(i, 8) for i in tqdm(range(num), position=0))
df_train_1 = pd.concat(results_train_1, axis=0)
df_train_1 = df_train_1.sort_values(['sym','date','time'])
df_train_1.to_parquet('./feature_data/df_train_8.parquet')

100%|██████████████████████████████████████████████████████████████████████████████████| 64/64 [08:14<00:00,  7.73s/it]


In [18]:
del results_train_1, df_train_1
gc.collect()

0

In [17]:
num = 64
results_train_1 = Parallel(n_jobs=-1)(delayed(gen_factor_train)(i, 9) for i in tqdm(range(num), position=0))
df_train_1 = pd.concat(results_train_1, axis=0)
df_train_1 = df_train_1.sort_values(['sym','date','time'])
df_train_1.to_parquet('./feature_data/df_train_9.parquet')

100%|██████████████████████████████████████████████████████████████████████████████████| 64/64 [10:57<00:00, 10.28s/it]


In [18]:
del results_train_1, df_train_1
gc.collect()

0

# 生成test的特征

In [15]:
def gen_factor_test(date, code):
    # 原始特征
    col_org = pd.read_csv(f'./data/test/snapshot_sym0_date64_am.csv', index_col=0).columns.to_list()
    # 生成的特征
    bollinger_fea = [f'{label}Bollinger{i}' for i in [5, 10, 20, 40] for label in ['upper', 'middle', 'lower']]
    cmo_fea = [f'com{i}' for i in [5, 10, 20, 40]]
    mom_fea = [f'mom{i}' for i in [1, 5, 10, 20, 40]]
    roc_fea = [f'roc{i}' for i in [1, 5, 10, 20, 40]]
    rsi_fea = [f'rsi{i}' for i in [5, 10, 20, 40]]
    mktIntensity_fea = [f'qs{i}' for i in range(1,6)] + [f'es{i}' for i in range(1,6)]
    elatricityTrading_fea = ['elatricityTrading']
    volatility_fea = [f'volatility{i}' for i in [5, 10, 20, 40]]
    mpc_fea = [f'mpc{i}' for i in [5, 10, 20, 40]]
    timeEncoding_fea = ['timeEncoding','hour','minute','second']
    # 需要被扩充的 生成的特征
    col_gen = bollinger_fea + cmo_fea + mom_fea + roc_fea + rsi_fea + mktIntensity_fea + elatricityTrading_fea + volatility_fea + mpc_fea
    # 算子
    operators = ['mom', 'mean', 'std', 'max', 'min', 'skew', 'kurt', 'qtlu', 'qtld', 'rank', 'imax', 'imin']
    # 扩充特征——原始的特征（不含sym,date,time）
    fea_org_expanded = [f'{f}_{op}{w}' for f in col_org[3:] for op in operators for w in [5, 10, 20, 40]]
    # 扩充特征——生成的特征
    fea_gen_expanded = [f'{f}_{op}{w}' for f in col_gen for op in operators for w in [5, 10, 20, 40]]
    # 所有被扩充的特征
    fac_to_remain = fea_org_expanded + fea_gen_expanded
    
    window = [5,10,20,40]
    
    # 处理csv不存在的情况
    df_am_all = pd.DataFrame()
    df_pm_all = pd.DataFrame()
    
    am_file = f'./data/test/snapshot_sym{code}_date{date}_am.csv'
    isExists = os.path.exists(am_file)
    if isExists:
        data_am = pd.read_csv(am_file, index_col = 0)
        factor_am0, time_am = cal_bollingerBands(data_am)
        factor_am1 = cal_cmo(data_am)
        factor_am2 = cal_mom(data_am)
        factor_am3 = cal_roc(data_am)
        factor_am4 = cal_rsi(data_am)
        factor_am5 = cal_mktIntensity(data_am)
        factor_am6 = cal_elatricityTrading(data_am)
        factor_am7  = cal_volatility(data_am)
        factor_am8  = cal_mpc(data_am)
        factor_am9 = cal_timeEncoding(data_am)
        
        factor_am = np.concatenate([factor_am0, factor_am1, factor_am2, factor_am3, factor_am4, factor_am5, factor_am6, factor_am7, factor_am8, factor_am9], axis = 1)
        factor_am = np.concatenate([data_am.values, factor_am],axis=1)
        
        df_am = pd.DataFrame(factor_am, columns=col_org+col_gen+timeEncoding_fea)
        df_am = df_am.set_index(['time','sym'])
        
        df_am_expanded = pd.DataFrame(index=df_am.index)
        for fea in df_am.columns:
            tmp = cal_extra_fea(df_am[fea], window=window, fac_to_remain=fac_to_remain)
            df_am_expanded = pd.concat([df_am_expanded,tmp], axis=1)
        df_am_all = pd.concat([df_am, df_am_expanded], axis=1)
        
        # 释放内存
        del data_am, factor_am, factor_am0, factor_am1, factor_am2, factor_am3, factor_am4, factor_am5, factor_am6, factor_am7, factor_am8, factor_am9
        gc.collect()
        
    pm_file = f'./data/test/snapshot_sym{code}_date{date}_pm.csv'
    isExists = os.path.exists(pm_file)
    if isExists:
        data_pm = pd.read_csv(pm_file, index_col=0)
        factor_pm0, time_pm = cal_bollingerBands(data_pm)
        factor_pm1 = cal_cmo(data_pm)
        factor_pm2 = cal_mom(data_pm)
        factor_pm3 = cal_roc(data_pm)
        factor_pm4 = cal_rsi(data_pm)
        factor_pm5 = cal_mktIntensity(data_pm)
        factor_pm6 = cal_elatricityTrading(data_pm)
        factor_pm7  = cal_volatility(data_pm)
        factor_pm8  = cal_mpc(data_pm)
        factor_pm9 = cal_timeEncoding(data_pm)
        
        factor_pm = np.concatenate([factor_pm0, factor_pm1, factor_pm2, factor_pm3, factor_pm4, factor_pm5, factor_pm6, factor_pm7, factor_pm8, factor_pm9], axis = 1)
        factor_pm = np.concatenate([data_pm.values, factor_pm],axis=1)
        
        df_pm = pd.DataFrame(factor_pm, columns=col_org+col_gen+timeEncoding_fea)
        df_pm = df_pm.set_index(['time','sym'])
        
        df_pm_expanded = pd.DataFrame(index=df_pm.index)
        for fea in df_pm.columns:
            tmp = cal_extra_fea(df_pm[fea], window=window, fac_to_remain=fac_to_remain)
            df_pm_expanded = pd.concat([df_pm_expanded,tmp], axis=1)
        df_pm_all = pd.concat([df_pm, df_pm_expanded], axis=1)
        
        # 释放内存
        del data_pm, factor_pm, factor_pm0, factor_pm1, factor_pm2, factor_pm3, factor_pm4, factor_pm5, factor_pm6, factor_pm7, factor_pm8, factor_pm9
        gc.collect()

    factor = pd.concat([df_am_all, df_pm_all])

    return factor.reset_index()

In [20]:
num = range(64, 79)
results_test_1 = Parallel(n_jobs=-1)(delayed(gen_factor_test)(i, j) for i in tqdm(num, position=0) for j in range(0,2))
df_test_1 = pd.concat(results_test_1, axis=0)
df_test_1 = df_test_1.sort_values(['sym','date','time'])
df_test_1.to_parquet('./feature_data/df_test_1.parquet')

100%|████████████████████████████████████████████████████████████████████████████████| 15/15 [00:00<00:00, 4619.96it/s]


In [21]:
del results_test_1, df_test_1
gc.collect()

0

In [22]:
num = range(64, 79)
results_test_1 = Parallel(n_jobs=-1)(delayed(gen_factor_test)(i, j) for i in tqdm(num, position=0) for j in range(2,4))
df_test_1 = pd.concat(results_test_1, axis=0)
df_test_1 = df_test_1.sort_values(['sym','date','time'])
df_test_1.to_parquet('./feature_data/df_test_2.parquet')

100%|██████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:00<?, ?it/s]


In [23]:
del results_test_1, df_test_1
gc.collect()

0

In [16]:
num = range(64, 79)
results_test_1 = Parallel(n_jobs=-1)(delayed(gen_factor_test)(i, j) for i in tqdm(num, position=0) for j in range(4,6))
df_test_1 = pd.concat(results_test_1, axis=0)
df_test_1 = df_test_1.sort_values(['sym','date','time'])
df_test_1.to_parquet('./feature_data/df_test_3.parquet')

100%|█████████████████████████████████████████████████████████████████████████████████| 15/15 [00:00<00:00, 151.75it/s]


In [25]:
del results_test_1, df_test_1
gc.collect()

0

In [26]:
num = range(64, 79)
results_test_1 = Parallel(n_jobs=-1)(delayed(gen_factor_test)(i, j) for i in tqdm(num, position=0) for j in range(6,8))
df_test_1 = pd.concat(results_test_1, axis=0)
df_test_1 = df_test_1.sort_values(['sym','date','time'])
df_test_1.to_parquet('./feature_data/df_test_4.parquet')

100%|██████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:00<?, ?it/s]


In [27]:
del results_test_1, df_test_1
gc.collect()

0

In [28]:
num = range(64, 79)
results_test_1 = Parallel(n_jobs=-1)(delayed(gen_factor_test)(i, j) for i in tqdm(num, position=0) for j in range(8,10))
df_test_1 = pd.concat(results_test_1, axis=0)
df_test_1 = df_test_1.sort_values(['sym','date','time'])
df_test_1.to_parquet('./feature_data/df_test_5.parquet')

100%|██████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:00<?, ?it/s]


In [29]:
del results_test_1, df_test_1
gc.collect()

0

# 处理原始train

## sort; int; encoding

In [2]:
df_train_6 = pd.read_parquet('./feature_data/df_train_9.parquet')

In [3]:
print(df_train_1.shape)

(483758, 3540)


In [15]:
print(df_train_2.shape)

(503748, 3540)


In [24]:
print(df_train_3.shape)

(465767, 3540)


In [3]:
print(df_train_4.shape)

(487756, 3541)


In [4]:
print(df_train_5.shape)

(251874, 3540)


In [3]:
print(df_train_6.shape)

(255872, 3540)


In [10]:
df_train_6[['timeEncoding','hour','minute','second','sym','date']]

,timeEncoding,hour,minute,second,sym,date
0,0,9,40,3,9,0
1,0,9,40,6,9,0
2,0,9,40,9,9,0
3,0,9,40,12,9,0
4,0,9,40,15,9,0
...,...,...,...,...,...,...
255867,7,14,49,45,9,63
255868,7,14,49,48,9,63
255869,7,14,49,51,9,63
255870,7,14,49,54,9,63


In [5]:
df_train_6[['hour','minute','second']] = df_train_6[['hour','minute','second']].astype('int')

In [6]:
time = pd.to_datetime(df_train_6['time'])
trading_time_intervals = [
    (pd.to_datetime('09:30:00').time(), pd.to_datetime('10:00:00').time()),
    (pd.to_datetime('10:00:00').time(), pd.to_datetime('10:30:00').time()),
    (pd.to_datetime('10:30:00').time(), pd.to_datetime('11:00:00').time()),
    (pd.to_datetime('11:00:00').time(), pd.to_datetime('11:30:00').time()),
    (pd.to_datetime('13:00:00').time(), pd.to_datetime('13:30:00').time()),
    (pd.to_datetime('13:30:00').time(), pd.to_datetime('14:00:00').time()),
    (pd.to_datetime('14:00:00').time(), pd.to_datetime('14:30:00').time()),
    (pd.to_datetime('14:30:00').time(), pd.to_datetime('15:00:00').time())
]
trading_intervals_encoding = np.zeros((len(df_train_6),), dtype=int)
for idx, (start_time, end_time) in enumerate(trading_time_intervals):
    mask = (time.dt.time >= start_time) & (time.dt.time < end_time)
    trading_intervals_encoding[mask] = idx
    
df_train_6['timeEncoding'] = trading_intervals_encoding.reshape(-1,1)

In [7]:
df_train_6.dtypes

time             object
sym               int64
date              int64
n_close         float64
amount_delta    float64
                 ...   
mpc40_qtlu40    float64
mpc40_qtld40    float64
mpc40_rank40    float64
mpc40_imax40    float64
mpc40_imin40    float64
Length: 3540, dtype: object

In [8]:
df_train_6 = df_train_6.sort_values(['sym','date','time'])
df_train_6.reset_index(drop=True, inplace=True)

In [9]:
print(df_train_6.shape)

(255872, 3540)


In [11]:
df_train_6

,time,sym,date,n_close,amount_delta,n_midprice,n_bid1,n_bsize1,n_bid2,n_bsize2,...,mpc40_std40,mpc40_max40,mpc40_min40,mpc40_skew40,mpc40_kurt40,mpc40_qtlu40,mpc40_qtld40,mpc40_rank40,mpc40_imax40,mpc40_imin40
0,09:40:03,9,0,-0.000535,442953.0,-0.000446,-0.000535,6.564465e-08,-0.000713,3.200177e-06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,09:40:06,9,0,-0.000178,3257370.0,-0.000267,-0.000357,1.641116e-08,-0.000713,9.299658e-07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,09:40:09,9,0,-0.000713,2107543.0,-0.000980,-0.001248,3.282232e-07,-0.001426,1.914636e-07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,09:40:12,9,0,-0.000357,1469514.0,-0.000267,-0.000357,9.846697e-08,-0.001248,5.087460e-07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,09:40:15,9,0,-0.000357,3254794.0,-0.000446,-0.000535,1.094077e-07,-0.000713,8.752620e-08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
255867,14:49:45,9,63,0.007817,89343.0,0.007879,0.007817,2.406970e-07,0.007693,1.196264e-06,...,0.046088,0.194175,0.034783,-0.213406,-0.849698,0.154228,0.069565,0.3500,0.775,0.225
255868,14:49:48,9,63,0.007941,706651.0,0.007879,0.007817,5.470387e-09,0.007693,1.157972e-06,...,0.045641,0.194175,0.034783,-0.142536,-0.801970,0.150476,0.069565,0.3625,0.800,0.250
255869,14:49:51,9,63,0.008066,1555556.0,0.008003,0.007941,1.903695e-08,0.007817,3.501048e-07,...,0.045355,0.194175,0.034783,-0.105250,-0.755982,0.148123,0.069565,0.4500,0.825,0.275
255870,14:49:54,9,63,0.008066,227459.0,0.007941,0.007817,4.566679e-07,0.007693,5.743907e-07,...,0.045232,0.194175,0.034783,-0.064218,-0.735427,0.148123,0.069565,0.4500,0.850,0.300


In [12]:
df_train_6.to_parquet('./feature_data/df_train_6.parquet')

## 转float32

In [2]:
df_train_1 = pd.read_parquet('./feature_data/df_train_1.parquet')

In [3]:
df_train_1.shape

(483758, 3540)

In [4]:
df_train_1[['label_5','label_10','label_20','label_40','label_60']].dtypes

label_5     int64
label_10    int64
label_20    int64
label_40    int64
label_60    int64
dtype: object

In [3]:
cols_to_remove = ['timeEncoding','hour','minute','second','sym','date','time','label_5','label_10','label_20','label_40','label_60']
col_to_f32 = [col for col in df_train_1.columns.to_list() if col not in cols_to_remove]

In [9]:
df_train_1.shape[1]

3540

In [14]:
for i in tqdm(range(8)):
    col = col_to_f32[i*500:(i+1)*500]
    df_train_1[col].astype(np.float32).to_parquet(f'./feature_data_f32/df_train_1_{i}.parquet')

100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [02:23<00:00, 17.90s/it]


In [4]:
del df_train_1
gc.collect()

0

In [17]:
df_train_2 = pd.read_parquet('./feature_data/df_train_2.parquet')

In [18]:
df_train_2.shape[1]

3540

In [19]:
for i in tqdm(range(8)):
    col = col_to_f32[i*500:(i+1)*500]
    df_train_2[col].astype(np.float32).to_parquet(f'./feature_data_f32/df_train_2_{i}.parquet')

100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [02:24<00:00, 18.05s/it]


In [21]:
del df_train_2
gc.collect()

0

In [5]:
df_train_3 = pd.read_parquet('./feature_data/df_train_3.parquet')

In [6]:
df_train_3.shape[1]

3540

In [7]:
for i in tqdm(range(8)):
    col = col_to_f32[i*500:(i+1)*500]
    df_train_3[col].astype(np.float32).to_parquet(f'./feature_data_f32/df_train_3_{i}.parquet')

100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [02:25<00:00, 18.24s/it]


In [8]:
del df_train_3
gc.collect()

0

In [9]:
df_train_4 = pd.read_parquet('./feature_data/df_train_4.parquet')

In [10]:
df_train_4.shape[1]

3540

In [11]:
for i in tqdm(range(8)):
    col = col_to_f32[i*500:(i+1)*500]
    df_train_4[col].astype(np.float32).to_parquet(f'./feature_data_f32/df_train_4_{i}.parquet')

100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [02:38<00:00, 19.83s/it]


In [12]:
del df_train_4
gc.collect()

0

In [13]:
df_train_5 = pd.read_parquet('./feature_data/df_train_5.parquet')

In [14]:
df_train_5.shape[1]

3540

In [15]:
for i in tqdm(range(8)):
    col = col_to_f32[i*500:(i+1)*500]
    df_train_5[col].astype(np.float32).to_parquet(f'./feature_data_f32/df_train_5_{i}.parquet')

100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [01:01<00:00,  7.69s/it]


In [16]:
del df_train_5
gc.collect()

0

In [17]:
df_train_6 = pd.read_parquet('./feature_data/df_train_6.parquet')

In [18]:
df_train_6.shape[1]

3540

In [19]:
for i in tqdm(range(8)):
    col = col_to_f32[i*500:(i+1)*500]
    df_train_6[col].astype(np.float32).to_parquet(f'./feature_data_f32/df_train_6_{i}.parquet')

100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [01:21<00:00, 10.18s/it]


In [20]:
del df_train_6
gc.collect()

0

## 拼接

In [7]:
for i in tqdm(range(1,7)):
    tmp = pd.read_parquet(f'./feature_data/df_train_{i}.parquet')
    df_train = tmp[['sym','date','time','label_5','label_10','label_20','label_40','label_60','timeEncoding','hour','minute','second']]
    del tmp
    gc.collect()
    for j in range(8):
        tmp1 = pd.read_parquet(f'./feature_data_f32/df_train_{i}_{j}.parquet')
        df_train = pd.concat([df_train, tmp1], axis=1)
        del tmp1
        gc.collect()
    df_train.to_parquet(f'./feature_data_f32/df_train_{i}.parquet')
    del df_train
    gc.collect()

100%|███████████████████████████████████████████████████████████████████████████████████| 6/6 [26:09<00:00, 261.66s/it]


In [2]:
df_train_1 = pd.read_parquet('./feature_data_f32/df_train_1.parquet')
df_train_2 = pd.read_parquet('./feature_data_f32/df_train_2.parquet')

In [3]:
df_train_tmp_1 = pd.concat([df_train_1, df_train_2], axis=0)

In [4]:
df_train_tmp_1.shape

(987506, 3540)

In [5]:
df_train_tmp_1.to_parquet(f'./feature_data_f32/df_train_tmp_1.parquet')

In [2]:
df_train_3 = pd.read_parquet('./feature_data_f32/df_train_3.parquet')
df_train_4 = pd.read_parquet('./feature_data_f32/df_train_4.parquet')

In [3]:
df_train_tmp_2 = pd.concat([df_train_3, df_train_4], axis=0)

In [4]:
df_train_tmp_2.shape

(953523, 3540)

In [5]:
df_train_tmp_2.to_parquet(f'./feature_data_f32/df_train_tmp_2.parquet')

In [2]:
df_train_5 = pd.read_parquet('./feature_data_f32/df_train_5.parquet')
df_train_6 = pd.read_parquet('./feature_data_f32/df_train_6.parquet')

In [3]:
df_train_tmp_3 = pd.concat([df_train_5, df_train_6], axis=0)

In [4]:
df_train_tmp_3.shape

(507746, 3540)

In [5]:
df_train_tmp_3.to_parquet(f'./feature_data_f32/df_train_tmp_3.parquet')

## 划分拆分

In [56]:
df_train_tmp_1 = pd.read_parquet(f'./feature_data_f32/df_train_tmp_3.parquet')

In [70]:
sym0_date0_20 = df_train_tmp_1[(df_train_tmp_1['sym']==9) & (df_train_tmp_1['date']<=20)]

In [71]:
sym0_date0_20.reset_index(drop=True, inplace=True)

In [72]:
sym0_date0_20.dtypes

sym               int64
date              int64
time             object
label_5           int64
label_10          int64
                 ...   
mpc40_qtlu40    float32
mpc40_qtld40    float32
mpc40_rank40    float32
mpc40_imax40    float32
mpc40_imin40    float32
Length: 3540, dtype: object

In [74]:
sym0_date0_20.to_parquet(f'./feature_data_f32/20/sym9_date0_20.parquet')

In [73]:
sym0_date0_20

,sym,date,time,label_5,label_10,label_20,label_40,label_60,timeEncoding,hour,...,mpc40_std40,mpc40_max40,mpc40_min40,mpc40_skew40,mpc40_kurt40,mpc40_qtlu40,mpc40_qtld40,mpc40_rank40,mpc40_imax40,mpc40_imin40
0,9,0,09:40:03,1,1,1,1,1,0,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,9,0,09:40:06,1,1,1,1,1,0,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,9,0,09:40:09,2,2,1,2,1,0,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,9,0,09:40:12,0,1,1,1,1,0,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,9,0,09:40:15,0,1,1,1,1,0,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83953,9,20,14:49:45,1,1,1,1,0,7,14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
83954,9,20,14:49:48,1,1,1,1,0,7,14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
83955,9,20,14:49:51,1,1,1,1,0,7,14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
83956,9,20,14:49:54,1,1,1,1,0,7,14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [75]:
sym0_date21_40 = df_train_tmp_1[(df_train_tmp_1['sym']==9) & (df_train_tmp_1['date']<=40) & (df_train_tmp_1['date']>20)]

In [76]:
sym0_date21_40.reset_index(drop=True, inplace=True)

In [78]:
sym0_date21_40.to_parquet(f'./feature_data_f32/40/sym9_date21_40.parquet')

In [77]:
sym0_date21_40

,sym,date,time,label_5,label_10,label_20,label_40,label_60,timeEncoding,hour,...,mpc40_std40,mpc40_max40,mpc40_min40,mpc40_skew40,mpc40_kurt40,mpc40_qtlu40,mpc40_qtld40,mpc40_rank40,mpc40_imax40,mpc40_imin40
0,9,21,09:40:03,0,0,0,0,0,0,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,9,21,09:40:06,1,1,1,1,0,0,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,9,21,09:40:09,1,1,1,1,0,0,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,9,21,09:40:12,1,1,1,1,1,0,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,9,21,09:40:15,1,1,1,1,1,0,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79955,9,40,14:49:45,1,1,1,1,0,7,14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79956,9,40,14:49:48,1,1,1,1,0,7,14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79957,9,40,14:49:51,1,1,1,1,0,7,14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79958,9,40,14:49:54,1,1,1,1,0,7,14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [79]:
sym0_date41_63 = df_train_tmp_1[(df_train_tmp_1['sym']==9) & (df_train_tmp_1['date']<=63) & (df_train_tmp_1['date']>40)]

In [80]:
sym0_date41_63.reset_index(drop=True, inplace=True)

In [81]:
sym0_date41_63.to_parquet(f'./feature_data_f32/63/sym9_date41_63.parquet')

In [82]:
sym0_date41_63

,sym,date,time,label_5,label_10,label_20,label_40,label_60,timeEncoding,hour,...,mpc40_std40,mpc40_max40,mpc40_min40,mpc40_skew40,mpc40_kurt40,mpc40_qtlu40,mpc40_qtld40,mpc40_rank40,mpc40_imax40,mpc40_imin40
0,9,41,09:40:03,0,0,0,0,0,0,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,9,41,09:40:06,1,0,0,0,0,0,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,9,41,09:40:09,1,0,0,0,0,0,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,9,41,09:40:12,0,0,0,0,0,0,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,9,41,09:40:15,1,0,0,0,0,0,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91949,9,63,14:49:45,1,1,1,1,1,7,14,...,0.046088,0.194175,0.034783,-0.213406,-0.849698,0.154228,0.069565,0.3500,0.775,0.225
91950,9,63,14:49:48,1,1,1,1,1,7,14,...,0.045641,0.194175,0.034783,-0.142536,-0.801970,0.150476,0.069565,0.3625,0.800,0.250
91951,9,63,14:49:51,1,1,1,1,1,7,14,...,0.045355,0.194175,0.034783,-0.105250,-0.755982,0.148123,0.069565,0.4500,0.825,0.275
91952,9,63,14:49:54,1,1,1,1,1,7,14,...,0.045232,0.194175,0.034783,-0.064218,-0.735427,0.148123,0.069565,0.4500,0.850,0.300


In [2]:
date0_20 = pd.DataFrame()
for i in tqdm(range(10)):
    tmp = pd.read_parquet(f'./feature_data_f32/20/sym{i}_date0_20.parquet')
    date0_20 = pd.concat([date0_20,tmp], axis=0)
    del tmp
    gc.collect()

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [02:06<00:00, 12.66s/it]


In [5]:
date0_20.shape

(777611, 3540)

In [3]:
date0_20.dtypes

sym               int64
date              int64
time             object
label_5           int64
label_10          int64
                 ...   
mpc40_qtlu40    float32
mpc40_qtld40    float32
mpc40_rank40    float32
mpc40_imax40    float32
mpc40_imin40    float32
Length: 3540, dtype: object

In [6]:
date0_20.to_parquet(f'./feature_data_f32/df_train_date0_20.parquet')

In [4]:
date0_20

,sym,date,time,label_5,label_10,label_20,label_40,label_60,timeEncoding,hour,...,mpc40_std40,mpc40_max40,mpc40_min40,mpc40_skew40,mpc40_kurt40,mpc40_qtlu40,mpc40_qtld40,mpc40_rank40,mpc40_imax40,mpc40_imin40
0,0,0,09:40:03,1,0,0,0,0,0,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,0,09:40:06,0,0,0,0,0,0,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,0,09:40:09,1,0,0,0,0,0,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,0,09:40:12,0,0,0,0,0,0,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,0,09:40:15,1,0,0,0,0,0,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83953,9,20,14:49:45,1,1,1,1,0,7,14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
83954,9,20,14:49:48,1,1,1,1,0,7,14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
83955,9,20,14:49:51,1,1,1,1,0,7,14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
83956,9,20,14:49:54,1,1,1,1,0,7,14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
date21_40 = pd.DataFrame()
for i in tqdm(range(10)):
    tmp = pd.read_parquet(f'./feature_data_f32/40/sym{i}_date21_40.parquet')
    date21_40 = pd.concat([date21_40,tmp], axis=0)
    del tmp
    gc.collect()

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [02:23<00:00, 14.36s/it]


In [3]:
date21_40.shape

(795602, 3540)

In [4]:
date21_40.dtypes

sym               int64
date              int64
time             object
label_5           int64
label_10          int64
                 ...   
mpc40_qtlu40    float32
mpc40_qtld40    float32
mpc40_rank40    float32
mpc40_imax40    float32
mpc40_imin40    float32
Length: 3540, dtype: object

In [6]:
date21_40.to_parquet(f'./feature_data_f32/df_train_date21_40.parquet')

In [5]:
date21_40

,sym,date,time,label_5,label_10,label_20,label_40,label_60,timeEncoding,hour,...,mpc40_std40,mpc40_max40,mpc40_min40,mpc40_skew40,mpc40_kurt40,mpc40_qtlu40,mpc40_qtld40,mpc40_rank40,mpc40_imax40,mpc40_imin40
0,0,21,09:40:03,1,0,0,0,0,0,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,21,09:40:06,1,0,0,0,0,0,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,21,09:40:09,0,0,0,0,0,0,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,21,09:40:12,0,0,0,0,0,0,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,21,09:40:15,0,0,0,0,0,0,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79955,9,40,14:49:45,1,1,1,1,0,7,14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79956,9,40,14:49:48,1,1,1,1,0,7,14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79957,9,40,14:49:51,1,1,1,1,0,7,14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79958,9,40,14:49:54,1,1,1,1,0,7,14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
date41_63 = pd.DataFrame()
for i in tqdm(range(10)):
    tmp = pd.read_parquet(f'./feature_data_f32/63/sym{i}_date41_63.parquet')
    date41_63 = pd.concat([date41_63,tmp], axis=0)
    del tmp
    gc.collect()

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [02:44<00:00, 16.50s/it]


In [3]:
date41_63.shape

(875562, 3540)

In [4]:
date41_63.dtypes

sym               int64
date              int64
time             object
label_5           int64
label_10          int64
                 ...   
mpc40_qtlu40    float32
mpc40_qtld40    float32
mpc40_rank40    float32
mpc40_imax40    float32
mpc40_imin40    float32
Length: 3540, dtype: object

In [6]:
date41_63.to_parquet(f'./feature_data_f32/df_train_date41_63.parquet')

In [5]:
date41_63

,sym,date,time,label_5,label_10,label_20,label_40,label_60,timeEncoding,hour,...,mpc40_std40,mpc40_max40,mpc40_min40,mpc40_skew40,mpc40_kurt40,mpc40_qtlu40,mpc40_qtld40,mpc40_rank40,mpc40_imax40,mpc40_imin40
0,0,41,09:40:03,1,0,0,0,0,0,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,41,09:40:06,1,1,0,0,0,0,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,41,09:40:09,1,0,0,0,0,0,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,41,09:40:12,1,0,0,0,0,0,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,41,09:40:15,1,0,0,0,0,0,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91949,9,63,14:49:45,1,1,1,1,1,7,14,...,0.046088,0.194175,0.034783,-0.213406,-0.849698,0.154228,0.069565,0.3500,0.775,0.225
91950,9,63,14:49:48,1,1,1,1,1,7,14,...,0.045641,0.194175,0.034783,-0.142536,-0.801970,0.150476,0.069565,0.3625,0.800,0.250
91951,9,63,14:49:51,1,1,1,1,1,7,14,...,0.045355,0.194175,0.034783,-0.105250,-0.755982,0.148123,0.069565,0.4500,0.825,0.275
91952,9,63,14:49:54,1,1,1,1,1,7,14,...,0.045232,0.194175,0.034783,-0.064218,-0.735427,0.148123,0.069565,0.4500,0.850,0.300


In [7]:
777611 + 795602  + 875562 

2448775

# 处理test

## sort; int; encoding

In [40]:
df_test_5 = pd.read_parquet('./feature_data/df_test_5.parquet')

In [46]:
df_test_5[['timeEncoding','hour','minute','second']]

,timeEncoding,hour,minute,second
0,0,9,40,3
1,0,9,40,6
2,0,9,40,9
3,0,9,40,12
4,0,9,40,15
...,...,...,...,...
3993,7,14,49,45
3994,7,14,49,48
3995,7,14,49,51
3996,7,14,49,54


In [42]:
df_test_5[['hour','minute','second']] = df_test_5[['hour','minute','second']].astype('int')

In [43]:
time = pd.to_datetime(df_test_5['time'])
trading_time_intervals = [
    (pd.to_datetime('09:30:00').time(), pd.to_datetime('10:00:00').time()),
    (pd.to_datetime('10:00:00').time(), pd.to_datetime('10:30:00').time()),
    (pd.to_datetime('10:30:00').time(), pd.to_datetime('11:00:00').time()),
    (pd.to_datetime('11:00:00').time(), pd.to_datetime('11:30:00').time()),
    (pd.to_datetime('13:00:00').time(), pd.to_datetime('13:30:00').time()),
    (pd.to_datetime('13:30:00').time(), pd.to_datetime('14:00:00').time()),
    (pd.to_datetime('14:00:00').time(), pd.to_datetime('14:30:00').time()),
    (pd.to_datetime('14:30:00').time(), pd.to_datetime('15:00:00').time())
]
trading_intervals_encoding = np.zeros((len(df_test_5),), dtype=int)
for idx, (start_time, end_time) in enumerate(trading_time_intervals):
    mask = (time.dt.time >= start_time) & (time.dt.time < end_time)
    trading_intervals_encoding[mask] = idx
    
df_test_5['timeEncoding'] = trading_intervals_encoding.reshape(-1,1)

In [44]:
df_test_5.dtypes

time             object
sym               int64
date              int64
n_close         float64
amount_delta    float64
                 ...   
mpc40_qtlu40    float64
mpc40_qtld40    float64
mpc40_rank40    float64
mpc40_imax40    float64
mpc40_imin40    float64
Length: 3535, dtype: object

In [45]:
df_test_5

,time,sym,date,n_close,amount_delta,n_midprice,n_bid1,n_bsize1,n_bid2,n_bsize2,...,mpc40_std40,mpc40_max40,mpc40_min40,mpc40_skew40,mpc40_kurt40,mpc40_qtlu40,mpc40_qtld40,mpc40_rank40,mpc40_imax40,mpc40_imin40
0,09:40:03,8,64,-0.004695,9540.0,-0.004173,-0.004695,3.864733e-06,-0.005216,2.512076e-05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,09:40:06,8,64,-0.005216,207893.0,-0.004956,-0.005216,1.494363e-05,-0.005738,4.637679e-06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,09:40:09,8,64,-0.004695,11445.0,-0.004956,-0.005216,1.906601e-05,-0.005738,4.637679e-06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,09:40:12,8,64,-0.004695,7632.0,-0.004956,-0.005216,1.906601e-05,-0.005738,4.637679e-06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,09:40:15,8,64,-0.004695,0.0,-0.004956,-0.005216,1.906601e-05,-0.005738,5.925923e-06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3993,14:49:45,9,78,-0.001504,2305964.0,-0.001562,-0.001620,1.455123e-06,-0.001736,1.192544e-06,...,0.137856,-0.000000,-0.711111,1.751039,4.577189,-0.444049,-0.63013,1.0000,0.025,0.700
3994,14:49:48,9,78,-0.001620,3132273.0,-0.001678,-0.001736,8.971435e-07,-0.001851,9.297470e-07,...,0.160629,-0.000000,-0.711111,1.884315,4.079560,-0.444049,-0.63013,0.9875,0.050,0.725
3995,14:49:51,9,78,-0.001736,2364061.0,-0.001793,-0.001851,7.601650e-07,-0.001967,4.069968e-06,...,0.185259,0.068966,-0.711111,1.901927,3.451461,-0.444049,-0.63013,1.0000,0.025,0.750
3996,14:49:54,9,78,-0.001851,20587972.0,-0.002256,-0.002661,3.260351e-08,-0.002777,4.321606e-07,...,0.228848,0.344828,-0.711111,2.168181,4.691121,-0.444049,-0.63013,1.0000,0.025,0.775


In [47]:
df_test_5.to_parquet('./feature_data/df_test_5.parquet')

## 转float32

In [2]:
df_test_1 = pd.read_parquet('./feature_data/df_test_1.parquet')

In [3]:
cols_to_remove = ['timeEncoding','hour','minute','second','sym','date','time']
col_to_f32 = [col for col in df_test_1.columns.to_list() if col not in cols_to_remove]

In [4]:
df_test_1.shape[1]

3535

In [5]:
for i in tqdm(range(8)):
    col = col_to_f32[i*500:(i+1)*500]
    df_test_1[col].astype(np.float32).to_parquet(f'./feature_data_f32/df_test_1_{i}.parquet')

100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:30<00:00,  3.80s/it]


In [6]:
del df_test_1
gc.collect()

0

In [7]:
df_test_2 = pd.read_parquet('./feature_data/df_test_2.parquet')

In [8]:
df_test_2.shape[1]

3535

In [9]:
for i in tqdm(range(8)):
    col = col_to_f32[i*500:(i+1)*500]
    df_test_2[col].astype(np.float32).to_parquet(f'./feature_data_f32/df_test_2_{i}.parquet')

100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:29<00:00,  3.73s/it]


In [10]:
del df_test_2
gc.collect()

0

In [11]:
df_test_3 = pd.read_parquet('./feature_data/df_test_3.parquet')

In [12]:
df_test_3.shape[1]

3535

In [13]:
for i in tqdm(range(8)):
    col = col_to_f32[i*500:(i+1)*500]
    df_test_3[col].astype(np.float32).to_parquet(f'./feature_data_f32/df_test_3_{i}.parquet')

100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:28<00:00,  3.55s/it]


In [14]:
del df_test_3
gc.collect()

0

In [15]:
df_test_4 = pd.read_parquet('./feature_data/df_test_4.parquet')

In [16]:
df_test_4.shape[1]

3535

In [17]:
for i in tqdm(range(8)):
    col = col_to_f32[i*500:(i+1)*500]
    df_test_4[col].astype(np.float32).to_parquet(f'./feature_data_f32/df_test_4_{i}.parquet')

100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:29<00:00,  3.70s/it]


In [18]:
del df_test_4
gc.collect()

0

In [19]:
df_test_5 = pd.read_parquet('./feature_data/df_test_5.parquet')

In [20]:
df_test_5.shape[1]

3535

In [21]:
for i in tqdm(range(8)):
    col = col_to_f32[i*500:(i+1)*500]
    df_test_5[col].astype(np.float32).to_parquet(f'./feature_data_f32/df_test_5_{i}.parquet')

100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:31<00:00,  4.00s/it]


In [22]:
del df_test_5
gc.collect()

0

## 拼接 & 加label

In [2]:
for i in tqdm(range(1,6)):
    tmp = pd.read_parquet(f'./feature_data/df_test_{i}.parquet')
    df_test = tmp[['sym','date','time','timeEncoding','hour','minute','second']]
    del tmp
    gc.collect()
    for j in range(8):
        tmp1 = pd.read_parquet(f'./feature_data_f32/df_test_{i}_{j}.parquet')
        df_test = pd.concat([df_test, tmp1], axis=1)
        del tmp1
        gc.collect()
    df_test.to_parquet(f'./feature_data_f32/df_test_{i}.parquet')
    del df_test
    gc.collect()

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [03:12<00:00, 38.42s/it]


In [3]:
df_test = pd.DataFrame()
for i in tqdm(range(1,6)):
    tmp = pd.read_parquet(f'./feature_data_f32/df_test_{i}.parquet')
    df_test = pd.concat([df_test, tmp], axis=0)

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:59<00:00, 11.88s/it]


In [4]:
df_test.shape

(591704, 3535)

In [5]:
df_test

,sym,date,time,timeEncoding,hour,minute,second,n_close,amount_delta,n_midprice,...,mpc40_std40,mpc40_max40,mpc40_min40,mpc40_skew40,mpc40_kurt40,mpc40_qtlu40,mpc40_qtld40,mpc40_rank40,mpc40_imax40,mpc40_imin40
0,0,64,09:40:03,0,9,40,3,0.075679,6954413.0,0.075775,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,64,09:40:06,0,9,40,6,0.076256,13722908.0,0.076160,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,64,09:40:09,0,9,40,9,0.077027,21771348.0,0.077123,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,64,09:40:12,0,9,40,12,0.077027,2925847.0,0.077508,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,64,09:40:15,0,9,40,15,0.078375,9831626.0,0.078182,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3993,9,78,14:49:45,7,14,49,45,-0.001504,2305964.0,-0.001562,...,0.137856,-0.000000,-0.711111,1.751040,4.577188,-0.444049,-0.63013,1.0000,0.025,0.700
3994,9,78,14:49:48,7,14,49,48,-0.001620,3132273.0,-0.001678,...,0.160629,-0.000000,-0.711111,1.884315,4.079560,-0.444049,-0.63013,0.9875,0.050,0.725
3995,9,78,14:49:51,7,14,49,51,-0.001736,2364061.0,-0.001793,...,0.185259,0.068966,-0.711111,1.901927,3.451461,-0.444049,-0.63013,1.0000,0.025,0.750
3996,9,78,14:49:54,7,14,49,54,-0.001851,20587972.0,-0.002256,...,0.228848,0.344828,-0.711111,2.168181,4.691121,-0.444049,-0.63013,1.0000,0.025,0.775


In [6]:
df_test.to_parquet('./feature_data_f32/df_test.parquet')

In [2]:
test_label = pd.read_parquet('./feature_data_f32/test_label.parquet')

In [3]:
test_label = test_label.sort_values(['sym','date','time'])

In [4]:
test_label.reset_index(drop=True, inplace=True)

In [5]:
test_label.dtypes

sym          int64
date         int64
time        object
label_5      Int64
label_10     Int64
label_20     Int64
label_40     Int64
label_60     Int64
dtype: object

In [6]:
df_test = pd.read_parquet('./feature_data_f32/df_test.parquet')

In [7]:
df_test = df_test.sort_values(['sym','date','time'])

In [8]:
df_test.reset_index(drop=True, inplace=True)

In [9]:
df_test.shape

(591704, 3535)

In [10]:
df_test[['timeEncoding','hour','minute','second','sym','date','time']].dtypes

timeEncoding     int32
hour             int32
minute           int32
second           int32
sym              int64
date             int64
time            object
dtype: object

In [11]:
df_test

,sym,date,time,timeEncoding,hour,minute,second,n_close,amount_delta,n_midprice,...,mpc40_std40,mpc40_max40,mpc40_min40,mpc40_skew40,mpc40_kurt40,mpc40_qtlu40,mpc40_qtld40,mpc40_rank40,mpc40_imax40,mpc40_imin40
0,0,64,09:40:03,0,9,40,3,0.075679,6954413.0,0.075775,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,64,09:40:06,0,9,40,6,0.076256,13722908.0,0.076160,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,64,09:40:09,0,9,40,9,0.077027,21771348.0,0.077123,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,64,09:40:12,0,9,40,12,0.077027,2925847.0,0.077508,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,64,09:40:15,0,9,40,15,0.078375,9831626.0,0.078182,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
591699,9,78,14:49:45,7,14,49,45,-0.001504,2305964.0,-0.001562,...,0.137856,-0.000000,-0.711111,1.751040,4.577188,-0.444049,-0.63013,1.0000,0.025,0.700
591700,9,78,14:49:48,7,14,49,48,-0.001620,3132273.0,-0.001678,...,0.160629,-0.000000,-0.711111,1.884315,4.079560,-0.444049,-0.63013,0.9875,0.050,0.725
591701,9,78,14:49:51,7,14,49,51,-0.001736,2364061.0,-0.001793,...,0.185259,0.068966,-0.711111,1.901927,3.451461,-0.444049,-0.63013,1.0000,0.025,0.750
591702,9,78,14:49:54,7,14,49,54,-0.001851,20587972.0,-0.002256,...,0.228848,0.344828,-0.711111,2.168181,4.691121,-0.444049,-0.63013,1.0000,0.025,0.775


In [12]:
df_test = pd.concat([df_test, test_label[['label_5','label_10','label_20','label_40','label_60']]], axis=1)

In [13]:
df_test[['timeEncoding','hour','minute','second','sym','date','time','label_5','label_10','label_20','label_40','label_60']].dtypes

timeEncoding     int32
hour             int32
minute           int32
second           int32
sym              int64
date             int64
time            object
label_5          Int64
label_10         Int64
label_20         Int64
label_40         Int64
label_60         Int64
dtype: object

In [14]:
col1 = ['sym','date','time','label_5','label_10','label_20','label_40','label_60']
col2 = [col for col in df_test.columns.to_list() if col not in col1]
df_test_my = df_test[col1+col2]

In [16]:
df_test_my.dtypes

sym               int64
date              int64
time             object
label_5           Int64
label_10          Int64
                 ...   
mpc40_qtlu40    float32
mpc40_qtld40    float32
mpc40_rank40    float32
mpc40_imax40    float32
mpc40_imin40    float32
Length: 3540, dtype: object

In [17]:
df_test_my.shape

(591704, 3540)

In [15]:
df_test_my

,sym,date,time,label_5,label_10,label_20,label_40,label_60,timeEncoding,hour,...,mpc40_std40,mpc40_max40,mpc40_min40,mpc40_skew40,mpc40_kurt40,mpc40_qtlu40,mpc40_qtld40,mpc40_rank40,mpc40_imax40,mpc40_imin40
0,0,64,09:40:03,2,2,2,2,2,0,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,64,09:40:06,2,2,2,2,2,0,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,64,09:40:09,2,2,2,1,2,0,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,64,09:40:12,2,2,2,2,2,0,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,64,09:40:15,2,2,2,2,2,0,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
591699,9,78,14:49:45,<NA>,<NA>,<NA>,<NA>,<NA>,7,14,...,0.137856,-0.000000,-0.711111,1.751040,4.577188,-0.444049,-0.63013,1.0000,0.025,0.700
591700,9,78,14:49:48,<NA>,<NA>,<NA>,<NA>,<NA>,7,14,...,0.160629,-0.000000,-0.711111,1.884315,4.079560,-0.444049,-0.63013,0.9875,0.050,0.725
591701,9,78,14:49:51,<NA>,<NA>,<NA>,<NA>,<NA>,7,14,...,0.185259,0.068966,-0.711111,1.901927,3.451461,-0.444049,-0.63013,1.0000,0.025,0.750
591702,9,78,14:49:54,<NA>,<NA>,<NA>,<NA>,<NA>,7,14,...,0.228848,0.344828,-0.711111,2.168181,4.691121,-0.444049,-0.63013,1.0000,0.025,0.775


In [18]:
df_test_my.to_parquet('./feature_data_f32/df_test_my.parquet')

# 划分/拼接三个人的特征

In [ ]:
df_test_my.to_parquet('./feature_data_f32/df_test_my.parquet')
df_test_zxy.to_parquet('./feature_data_f32/df_test_zxy.parquet')
df_test_cyx.to_parquet('./feature_data_f32/df_test_cyx.parquet')